In [1]:
!uv add nltk

Resolved 118 packages in 1.64s
 Downloaded nltk
Prepared 2 packages in 1.10s
Installed 4 packages in 294ms
 + click==8.2.0
 + nltk==3.9.1
 + regex==2024.11.6
 + tqdm==4.67.1


In [ ]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error  # 루트 제곱 평균 오차
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import nltk #토큰화 불용어 처리

In [3]:
file_url = (
    "https://media.githubusercontent.com/media/musthave-ML10/data_source/main/spam.csv"
)

data =pd.read_csv(file_url)

data.shape


(5574, 2)

In [5]:
data.head()

,target,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
data["target"].unique()

array(['ham', 'spam'], dtype=object)

In [8]:
data["target"].value_counts()

target
ham     4827
spam     747
Name: count, dtype: int64

In [9]:
data.tail()

,target,text
5569,spam,This is the 2nd time we have tried 2 contact u...
5570,ham,Will ü b going to esplanade fr home?
5571,ham,"Pity, * was in mood for that. So...any other s..."
5572,ham,The guy did some bitching but I acted like i'd...
5573,ham,Rofl. Its true to its name


# 전처리 : 특수 기호 제거  

In [13]:
import string
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [14]:
sample_string = data["text"].loc[0]
sample_string

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [18]:
# for i in sample_string: #한글자씩 처리 
#     # print(i)
#     if i not in string.punctuation:
#         print(i)

In [21]:
"a" in "apple"


True

In [ ]:
new_string = []
for i in sample_string: #한글자씩 처리
    if i not in string.punctuation:
        new_string.append(i)

# print(new_string)
new_string = "".join(new_string)
new_string

['G', 'o', ' ', 'u', 'n', 't', 'i', 'l', ' ', 'j', 'u', 'r', 'o', 'n', 'g', ' ', 'p', 'o', 'i', 'n', 't', ' ', 'c', 'r', 'a', 'z', 'y', ' ', 'A', 'v', 'a', 'i', 'l', 'a', 'b', 'l', 'e', ' ', 'o', 'n', 'l', 'y', ' ', 'i', 'n', ' ', 'b', 'u', 'g', 'i', 's', ' ', 'n', ' ', 'g', 'r', 'e', 'a', 't', ' ', 'w', 'o', 'r', 'l', 'd', ' ', 'l', 'a', ' ', 'e', ' ', 'b', 'u', 'f', 'f', 'e', 't', ' ', 'C', 'i', 'n', 'e', ' ', 't', 'h', 'e', 'r', 'e', ' ', 'g', 'o', 't', ' ', 'a', 'm', 'o', 'r', 'e', ' ', 'w', 'a', 't']


'Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat'

In [39]:
def remove_punc(sample_string):
    new_string = []
    for i in sample_string:  # 한글자씩 처리
        if i not in string.punctuation:
            new_string.append(i)

    # print(new_string)
    new_string = "".join(new_string)
    return new_string

In [40]:
remove_punc(sample_string)

'Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat'

In [67]:
data["text"] = data["text"].apply(remove_punc)

In [69]:
data.head()

,target,text
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni
2,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say early hor u c already say
4,ham,nah i think goes usf lives around though


In [70]:
# data["text"].map(remove_punc)

# 전처리 :불용어 제거

In [71]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [72]:
from nltk.corpus import stopwords

In [73]:
len(stopwords.words("english"))

198

In [74]:
sample_string = data["text"].loc[0]
print(sample_string.split())

['go', 'jurong', 'point', 'crazy', 'available', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'got', 'amore', 'wat']


In [75]:
new_string = []
for i in sample_string.split():
    if i not in stopwords.words("english"):
        new_string.append(i.lower())
        
new_string = " ".join(new_string)
new_string

'go jurong point crazy available bugis n great world la e buffet cine got amore wat'

In [76]:
def stop_words(x):
    new_string = []
    for i in x.split():
        if i not in stopwords.words("english"):
            new_string.append(i.lower())

    new_string = " ".join(new_string)
    return new_string

In [77]:
data["text"] = data["text"].apply(stop_words)


In [78]:
data.head()

,target,text
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni
2,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say early hor u c already say
4,ham,nah think goes usf lives around though


# 전처리 : 목표 컬럼 (종속변수)